In [13]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.tools import tool
from pydantic import SecretStr
from langchain_core.runnables import RunnableConfig
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage


@tool
def get_weather(location: str) -> str:
    """Get the current weather for a given location."""
    # This is a placeholder implementation. Replace with actual weather fetching logic.
    return f"The current weather in {location} is sunny."


tools = [get_weather]
config: RunnableConfig = {"configurable": {"thread_id": 1}}


model = ChatOpenAI(
    base_url="http://localhost:8080/v1",
    api_key=SecretStr("dummy"),  # llama.cpp ignores this
    model="mistral-nemo",
)


agent = initialize_agent(
    tools=tools,
    llm=model,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)


def call_model(state: MessagesState):
    response = agent.invoke({"input": state["messages"]})
    return {"messages": state["messages"] + [AIMessage(content=response["output"])]}


workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config: RunnableConfig = {"configurable": {"thread_id": "1"}}
query = "Hi, my name is Deepesh, What's the weather like in San Francisco? Please call the tool and respond whatever it returns"
input_messages = [HumanMessage(content=query)]

for chunk, metadata in app.stream(
    {"messages": input_messages}, config=config, stream_mode="messages"
):
    if isinstance(chunk, AIMessage):
        print(chunk.content, end="", flush=True)



> Entering new AgentExecutor chain...
To answer this question, I need to get the current weather for San Francisco.

Action: get_weather

Action Input: "San Francisco"To answer this question, I need to get the current weather for San Francisco.

Action: get_weather

Action Input: "San Francisco"
Observation: The current weather in San Francisco is sunny.
Thought:I now know the final answer

Final Answer: The current weather in San Francisco is sunny.I now know the final answer

Final Answer: The current weather in San Francisco is sunny.

> Finished chain.
The current weather in San Francisco is sunny.